In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pandas_datareader.data import DataReader

In [ ]:
from matplotlib import rcParams

# Restore old behavior of rounding default axis ranges
rcParams['axes.autolimit_mode'] = 'round_numbers'
rcParams['axes.xmargin'] = 0
rcParams['axes.ymargin'] = 0

### Taylor Rule
The Taylor rule is an interest rate forecasting model to determine what interest rates should be in order to shift the economy toward stable prices and full employment proposed by John Taylor in 1993.
The original Taylor Rule is in the following form:

\begin{align}
i_{t}=\pi_{t}+0.5x_{t}+0.5*(\pi_{t}-2)+2
\end{align}

where $i_{t}$ is the federal funds rate, $\pi_{t}$ is inflation, and $x_{t}$ is the output gap, actual output less potential output, the amount the economy can sustainably produce when capital and labor are fully employed.

The (original) Taylor rule predicts that the FOMC will raise the federal funds rate (tighten monetary policy) by one-half percentage point:

(1) for each percentage point that inflation rises relative to the Fed’s target, assumed to be 2 percent; or

(2) for each percentage point that that output rises relative to its potential.

The Taylor rule also predicts that when inflation is at target and output is at potential (the output gap is zero), the FOMC will set the real federal funds rate at 2 percent—about its historical average.

Now, let's see how original Taylor fits to the actual federal funds rate. We will download following variables from Federal Reserve Bank of St. Louis Economic Database (FRED) https://fred.stlouisfed.org/.

- nominal Gross Domestic Product in billions of current dollars - `GDP`
- nominal Potential Gross Domestic Product in billions of current dollars - `NGDPPOT`
- Implicit Price Deflator for Gross Domestic Product - `GDPDEF`
- Personal Consumption Expenditures: Chain-type Price Index - `PCEPI`
- Core PCE, Personal Consumption Expenditures Excluding Food and Energy - `PCEPILFE`
- Federal Funds Effective Rate - `FEDFUNDS`


In [ ]:
fred = DataReader(['GDP', 'NGDPPOT', 'GDPDEF','PCEPI','PCEPILFE','FEDFUNDS'],
                  'fred', start='1993-01', end='2023-12')
fred = fred.resample('QS').mean()
fred = fred.dropna()
fred.tail()

Below we are constructing the series that we need to test original Taylor rule:
- `Output` is real GDP 
- `Potential Output` is the real potential GDP
- `Output Gap` is the log real GDP less log real potential GDP
- `Inflation` is annualized quarterly percentage change in GDP implicit price deflator
- `PCE Inflation` is annualized quarterly percentage change in PCE
- `Core PCE Inflation` is annualized quarterly percentage change in Core PCE


In [ ]:
dta = pd.DataFrame()

dta['Output'] = np.log(fred['GDP']*10**9/fred['GDPDEF']*100)
dta['Potential Output'] = np.log(fred['NGDPPOT']*10**9/fred['GDPDEF']*100)
dta['Output_Gap'] = 400*(dta['Output']-dta['Potential Output'])
dta['Inflation'] = 400*fred['GDPDEF'].pct_change()
dta['PCE_Inflation'] = 400*fred['PCEPI'].pct_change()
dta['Core_PCE_Inflation'] = 400*fred['PCEPILFE'].pct_change()
dta['FFR'] = fred['FEDFUNDS']

dta.tail()

Now we can use the original Taylor formula to predict the interest rate. We will use three different inflation index to calculate the predicted FFR:

In [ ]:
dta['Taylor FFR PCE']     = dta['PCE_Inflation'] + 0.5*dta['Output_Gap'] + 0.5*(dta['PCE_Inflation']-2) + 2
dta['Taylor FFR COREPCE'] = dta['Core_PCE_Inflation'] + 0.5*dta['Output_Gap'] + 0.5*(dta['Core_PCE_Inflation']-2) + 2

fig, axs = plt.subplots(2, 1, figsize=(20, 15), sharex=False, sharey=False)
dta[['FFR','Taylor FFR PCE']].plot(ax=axs[0], lw=2, style=['k-','r--'])
dta[['FFR','Taylor FFR COREPCE']].plot(ax=axs[1], lw=2, style=['k-','r--'])

axs[0].legend(["FED Funds Rate", "Taylor Rule using PCE"]);
axs[1].legend(["FED Funds Rate", "Taylor Rule using Core PCE"]);

plt.show()

It seems that original Taylor rule does not predict well the behaviour of the interest rates. Now, let's estimate the following Taylor rule:

\begin{align}
    i_{t} = \rho_{i}i_{t-1} + (1-\rho_{i})*(\phi_{\pi}(\pi_{t}-\pi^{*})+\phi_{x}x_{t})
\end{align}

In [ ]:
Taylor_PCE = smf.ols(formula='FFR ~ 0 + FFR.shift(1) + PCE_Inflation + Output_Gap', data=dta).fit()
Taylor_COREPCE = smf.ols(formula='FFR ~ 0 + FFR.shift(1) + Core_PCE_Inflation + Output_Gap', data=dta).fit()

Taylor_COREPCE.summary()

Let's recover the parameter values from Taylor_COREPCE model:

In [ ]:
phi_pi=Taylor_COREPCE.params['Core_PCE_Inflation']/(1-Taylor_COREPCE.params['FFR.shift(1)'])
phi_x=Taylor_COREPCE.params['Output_Gap']/(1-Taylor_COREPCE.params['FFR.shift(1)'])

print('')
print('phi_pi    =', phi_pi)
print('phi_x    =', phi_x)

In [ ]:
dta['Taylor_PCE_fitted'] = Taylor_PCE.fittedvalues
dta['Taylor_COREPCE_fitted'] = Taylor_COREPCE.fittedvalues

fig, axs = plt.subplots(2, 1, figsize=(20, 15), sharex=False, sharey=False)
dta[['FFR','Taylor_PCE_fitted']].plot(ax=axs[0], lw=2, style=['k-','r--'])
dta[['FFR','Taylor_COREPCE_fitted']].plot(ax=axs[1], lw=2, style=['k-','r--'])

axs[0].legend(["FED Funds Rate", "Taylor Rule using PCE"]);
axs[1].legend(["FED Funds Rate", "Taylor Rule using Core PCE"]);

plt.show()

## 3-equation New Keynesian Model

### The Economy
The representative household maximizes the following expected
discounted sum of utilities over possible paths of consumption and
labor:

\begin{align}
 & U_{t}=\mathrm{E}_{t}\sum_{k=0}^{\infty}\beta^{k}\left[\frac{C_{t+k}^{1-\sigma}}{1-\sigma}-\phi\frac{N_{t+k}^{1+\eta}}{1+\eta}\right]
\end{align}

The nominal period budget constraint is:

\begin{equation}
P_{t}C_{t}+B_{t}=W_{t}N_{t}+\left(1+i_{t-1}\right)B_{t-1}+P_{t}D_{t}
\end{equation}

The final output good is a CES aggregate of a continuum of intermediates:

\begin{equation}
Y_{t}=\left[\intop_{0}^{1}Y_{t}\left(i\right)^{\tfrac{\varepsilon-1}{\varepsilon}}di\right]^{\tfrac{\varepsilon}{\varepsilon-1}}
\end{equation}


Each monopolistic firm $i$ produces a differentiated variety and
hires a homogenous type of labor according to the linear production
function:

\begin{align}
Y_{t}\left(i\right)=A_{t}N_{t}\left(i\right)^{1-\alpha}
\end{align}

Following Calvo, we assume now that firms adjust their price infrequently and that
the opportunity to adjust follows an exogenous Poisson process. Each
period there is a constant probability ($1-\theta$) that the firm
will be able to adjust its price, independently of past history. The
expected time between price adjustments is therefore $\dfrac{1}{1-\theta}$.
If the law of large numbers holds this implies that the fraction of
retailers not setting prices at $t$ is $\theta$. The draw is independent
of history, so that we do not need to keep track of firms changing
prices over time.

The solutions to the above household and firm problems give the following three key equations of the New Keynesian model.

### New Keynesian Phillips Curve
\begin{align}
\pi_{t} & =\beta\mathrm{E}_{t}\pi_{t+1}+\kappa x_{t}
\end{align}

In the above equation:
\begin{align}
\text{Output gap:} \quad & x_{t}=y_{t}-y_{t}^{n}\\
\text{Output gap coefficient:} \quad & \kappa=\frac{\left(1-\theta\right)\left(1-\beta\theta\right)}{\theta}\left(\sigma+\eta\right)
\end{align}

The actual output and natural level of output are equal to:
\begin{align}
\text{Actual output:} \quad & y_{t} & =a_{t}+(1-\alpha)n_{t}\\
\text{Natural level of output} \quad & y_{t}^{n} & =\dfrac{1+\eta}{\sigma(1-\alpha)+\eta+\alpha}a_{t}
\end{align}

### New Keynesian IS
\begin{align}
x_{t} & =\mathrm{E}_{t}x_{t+1}-\frac{1}{\sigma}\left(i_{t}-\mathrm{E}_{t}\pi_{t+1}-r_{t}^{n}\right)
\end{align}

In the above equation:
\begin{align}
\text{Natural real interest rate:} \quad & r_{t}^{n} & =\rho+\sigma\left(\dfrac{1+\eta}{\sigma(1-\alpha)+\eta+\alpha}\mathrm{E}_{t}\triangle a_{t+1}\right)
\end{align}

The technology evolves according to:
\begin{align}
\text{TFP:} \quad & a_{t}=\rho_{a}a_{t-1}+\varepsilon_{a,t}
\end{align}

The devation of natural real interest rate from its steady-state is given by:
\begin{align}
\hat{r}_{t}^{n} & =-\sigma\left(\dfrac{1+\eta}{\sigma(1-\alpha)+\eta+\alpha}\right)(1-\rho_{a})a_{t}
\end{align}


### Monetary Policy Rule
\begin{align}
i_{t}=\phi_{\pi}\pi_{t}+\phi_{x}x_{t}+\nu_{t}
\end{align}

The monetary policy shock evolves according to:
\begin{align}
\text{Monetary Shock:} \quad & \nu_{t}=\rho_{\nu}\nu_{t-1}+\varepsilon_{R,t}
\end{align}

In the above equations, the parameters correspond to:

|Parameter | Justification	
|:---|:---|
$\alpha$ |	Share of capital in production (output)	
$\beta$	| Household discount factor		
$\sigma$ |	Risk aversion (Inverse of intertemporal elasticity of substitution)	
$\eta$	| Inverse Frisch-elasticity		
$\theta$	| quarterly probability of price rigidity		
$\kappa$	| Coefficient of output gap in NKPC		
$\phi_{\pi}$	| Response of monetary rule to inflation		
$\phi_{x}$	| Response of monetary rule to output gap	
$\rho_{a}$	| Coefficient in TFP AR(1) regression		
$\rho_{v}$	| Coefficient of monetary policy shock AR(1) regression		


In [ ]:
from Dynare import *

In [ ]:
var = 'Pi x i y r_r n p nu w y_n r_n a'
varexo = 'eps_R'

param_values = {'alppha':0.25, sy.symbols('beta'):0.99, 'siggma':2, 'eta':3.77, 'rho_nu':0.5,
                'theta':0.75, 'kappa':0.1717, 'phi_pi':1.25, 'phi_x':0.125, 'rho_a':0.9}

model = ('-Pi + betta*Pi(+1)+kappa*x',
         '-x + x(+1) -1/siggma*(i-Pi(+1)-r_n)',
         '-i + phi_pi*Pi+phi_x*x+nu',
         '-r_n + -siggma*(1+eta)/(siggma*(1-alppha)+eta+alppha)*(1-rho_a)*a',
         '-r_r + i-Pi(+1)',
         '-y_n + (1+eta)/(siggma*(1-alppha)+eta+alppha)*a',
         '-x + y-y_n',
         '-nu + rho_nu*nu(-1)+eps_R',
         '-a + rho_a*a(-1)',
         '-y + a+(1-alppha)*n',
         '-Pi + p-p(-1)',
         '-w+p+siggma*y+eta*n')

initval = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [ ]:
rbc = Dynare(var, varexo, param_values, model, initval)

In [ ]:
rbc.steady()

In [ ]:
rbc.stoch_simul(irf=20)

# Homework
The other important shocks in the New Keynesian Model are demand and cost push-up shocks. We can incorporate them into the 3-eq New Keynesian model in following way.

The New Keynesian IS (NKIS) with demand shock where demand shock follows AR(1) process is given by

\begin{align}
x_{t} & =\mathrm{E}_{t}x_{t+1}-\frac{1}{\sigma}\left(i_{t}-\mathrm{E}_{t}\pi_{t+1}-r_{t}^{n}\right)+u_{t}\\
u_{t} & = \rho_{u}u_{t-1}+\epsilon_{u,t}
\end{align}

The New Keynesian Phillips Curve (NKPC) with cost push-up shock where cost push-up shock follows AR(1) process is given by

\begin{align}
\pi_{t} & =\beta\mathrm{E}_{t}\pi_{t+1}+\kappa x_{t}+e{t}\\
e_{t} & = \rho_{e}e_{t-1}+\epsilon_{e,t}
\end{align}


**Task 1**: Insert the **demand shock** given in the above **New Keynesian IS equation** into the New Keynesian Model. Compute impulse response functions (IRFS) of the variables following a **demand shock**. Briefly comment on the responses of inflation, output gap, and nominal interest rate. 

In [ ]:
var = 'Pi x i y r_r n p u w y_n r_n a'
varexo = 'eps_u'

param_values = {'alppha':0.25, sy.symbols('beta'):0.99, 'siggma':2, 'eta':3.77, 'rho_u':0.75,
                'theta':0.75, 'kappa':0.1717, 'phi_pi':1.25, 'phi_x':0.125, 'rho_a':0.9}

model = ('-Pi + betta*Pi(+1)+kappa*x',
#         'Here goes the New Keynesian IS curve with demand shock',
         '-i + phi_pi*Pi+phi_x*x',
         '-r_n + -siggma*(1+eta)/(siggma*(1-alppha)+eta+alppha)*(1-rho_a)*a',
         '-r_r + i-Pi(+1)',
         '-y_n + (1+eta)/(siggma*(1-alppha)+eta+alppha)*a',
         '-x + y-y_n',
#         'Here goes the AR(1) demand shock',
         '-a + rho_a*a(-1)',
         '-y + a+(1-alppha)*n',
         '-Pi + p-p(-1)',
         '-w+p+siggma*y+eta*n')

initval = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

**Task 2**: Insert the **cost push-up shock** given in the above **NKPC equation** into the New Keynesian Model. Compute impulse response functions (IRFS) of the variables following a **cost push-up shock**. Briefly comment on the responses of inflation, output gap, and nominal interest rate. 

In [ ]:
var = 'Pi x i y r_r n p e w y_n r_n a'
varexo = 'eps_e'

param_values = {'alppha':0.25, sy.symbols('beta'):0.99, 'siggma':2, 'eta':3.77, 'rho_e':0.75,
                'theta':0.75, 'kappa':0.1717, 'phi_pi':1.25, 'phi_x':0.125, 'rho_a':0.9}

model = (#         'Here goes the NKPC with cost push-up shock',
         '-x +x(+1) -1/siggma*(i-Pi(+1)-r_n)',
         '-i + phi_pi*Pi+phi_x*x',
         '-r_n + -siggma*(1+eta)/(siggma*(1-alppha)+eta+alppha)*(1-rho_a)*a',
         '-r_r + i-Pi(+1)',
         '-y_n + (1+eta)/(siggma*(1-alppha)+eta+alppha)*a',
         '-x + y-y_n',
#         'Here goes the AR(1) cost push-up shock',
         '-a + rho_a*a(-1)',
         '-y + a+(1-alppha)*n',
         '-Pi + p-p(-1)',
         '-w+p+siggma*y+eta*n')

initval = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

# Optional: Unemployment and Labor Markets over the Business Cycle

We will take a look at the data on unemployment and functioning of labor markets in the United States

Below we are downloading two datasets. 

The first dataset, `fred`, contains monthly data on: 

- whether in a given month the US economy was in a recession state (`1`) or not (`0`) - `USREC`
- number of people in labor force in thousands - `CLF16OV`
- number of unemployed people in thousands - `UNEMPLOY`
- unemployment rate in percent - `UNRATE`
- number of job openings (vacancies) in thousands - `JTSJOL`
- job vacancy rate - `JTSJOR`

The second dataset, `hours`, contains quarterly data on:

- real GDP in billions of 2009 dollars - `GDPC1`
- total hours worked in the nonfarm business sector (index) - `HOANBS`
- average hours worked per employee in the nonfarm business sector (index) - `PRS85006023`
- number of employees in the nonfarm business sector (index) - `PRS85006013`

In [ ]:
start = '1945-01'
end   = '2021-06'

In [ ]:
# Get FRED data
fred = DataReader(['USREC', 'CLF16OV', 'UNEMPLOY', 'UNRATE', 'JTSJOL', 'JTSJOR'], 
                   'fred', start=start, end=end)

hours = DataReader(['GDPC1', 'HOANBS', 'PRS85006023', 'PRS85006013'], 
                    'fred', start=start, end=end)

Separate trend and cyclical components of GDP, hours and employment

In [ ]:
hp_cycle = pd.DataFrame()
hp_trend = pd.DataFrame()

cf_cycle = pd.DataFrame()
cf_trend = pd.DataFrame()

for col in hours.columns:
    hp_cycle[col], hp_trend[col] = sm.tsa.filters.hpfilter(100*np.log(hours[col]).dropna(), lamb=1600)
    cf_cycle[col], cf_trend[col] = sm.tsa.filters.cffilter(100*np.log(hours[col]).dropna(), low=6, high=32)

Compare cyclical components of total hours worked vs its components: hours per employee and number of employees

In [ ]:
hp_cycle.columns = ['Output','Total Hours','Hours per Employee','Employment']
cf_cycle.columns = ['Output','Total Hours','Hours per Employee','Employment']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))

cf_cycle[['Total Hours','Employment']].to_period('D').plot(ax=ax1, style=['k-','r-'], lw=2)

ylim = ax1.get_ylim()

ax1.hlines(0, cf_cycle.index[0], cf_cycle.index[-1], linewidth=0.5)
ax1.fill_between(fred.index, ylim[0], ylim[1], fred['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

l = ax1.legend(loc='upper right')
l.get_frame().set_linewidth(0)
l.get_frame().set_alpha(1)

cf_cycle[['Total Hours','Hours per Employee']].to_period('D').plot(ax=ax2, style=['k-','r-'], lw=2)

ylim = ax2.get_ylim()

ax2.hlines(0, cf_cycle.index[0], cf_cycle.index[-1], linewidth=0.5)
ax2.fill_between(fred.index, ylim[0], ylim[1], fred['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

l = ax2.legend(loc='upper right')
l.get_frame().set_linewidth(0)
l.get_frame().set_alpha(1)

# plt.savefig('Hours_CF.pdf', bbox_inches='tight', pad_inches=0.05)
plt.show()

Calculate the variance-covariance matrix of total hours worked and its components

In [ ]:
hp_cycle[['Total Hours','Employment','Hours per Employee']].cov()

## Constructing the vacancy rate time series

The statistics on job openings (vacancies) from the `JOLTS` program are available only starting from December 2000. However, there are data on `Help Wanted Index`, which were gathered by private companies. Thanks to the work of Regis Barnichon, we can use them.

In [ ]:
dta = fred.copy()
dta.tail()

In [ ]:
dta[['JTSJOR','UNRATE']]['2000-12':].plot(lw=2)
plt.legend(frameon=False)
plt.show()

Read in Regis Barnichon's Composite Help Wanted Index and join the two datasets

In [ ]:
hwi = pd.read_csv('data/HWI_index_old.txt', delimiter='\t', skiprows=5)

# Manage dates
dates = []
for date in hwi['Date ']:
    dates.append(date[-2:]+'-01-'+date[0:4])

hwi.index = pd.to_datetime(dates)
hwi.index.rename('DATE', inplace=True)

# Cleanup
hwi = hwi.drop('Date ', 1)
hwi.columns = ['HWI']
hwi.tail()

In [ ]:
# Join datasets
df = dta.join(hwi)
df.tail()

Adjust the index to observed levels and splice the data from two sources

In [ ]:
df['Vacancies'] = df['JTSJOL']['2014-01-01'] * df['HWI'] / df['HWI']['2014-01-01']
df['Vacancies']['2005-01-01':] = df['JTSJOL']['2005-01-01':]

df[['Vacancies','JTSJOL']].plot(lw=2)
plt.legend(frameon=False)
plt.show()

Construct time series for unemployment and vacancy rates

In [ ]:
df['Unemployment rate'] = 100 * df['UNEMPLOY'] / df['CLF16OV']
df['Vacancy rate'] = 100 * df['Vacancies'] / df['CLF16OV']

In [ ]:
fig, ax = plt.subplots()

df['Vacancy rate'].to_period('D').plot(ax=ax, style='k', lw=2)
df['Unemployment rate'].to_period('D').plot(ax=ax, style='r', lw=2)

ylim = ax.get_ylim()

ax.fill_between(fred.index, ylim[0], ylim[1], fred['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

l = ax.legend(loc='upper left')
l.get_frame().set_linewidth(0)
l.get_frame().set_alpha(1)

plt.title('US vacancy and unemployment rates (%)')
# plt.savefig('VU.pdf')
plt.show()

## Behavior of unemployment and vacancy rates in the United States

Below I plot the scatterplot of unemployment and vacancy rates, with colors reflecting different decades. 

The resulting negative relationship is known as the Beveridge curve

In [ ]:
dfq = df.resample('QS').mean()

plt.plot(dfq['Unemployment rate']['1950-01-01':'1959-12-01'], 
         dfq['Vacancy rate']['1950-01-01':'1959-12-01'], 'o-', label='1950s')
plt.plot(dfq['Unemployment rate']['1960-01-01':'1969-12-01'], 
         dfq['Vacancy rate']['1960-01-01':'1969-12-01'], 'o-', label='1960s')
plt.plot(dfq['Unemployment rate']['1970-01-01':'1979-12-01'], 
         dfq['Vacancy rate']['1970-01-01':'1979-12-01'], 'o-', label='1970s')
plt.plot(dfq['Unemployment rate']['1980-01-01':'1989-12-01'], 
         dfq['Vacancy rate']['1980-01-01':'1989-12-01'], 'o-', label='1980s')
plt.plot(dfq['Unemployment rate']['1990-01-01':'1999-12-01'], 
         dfq['Vacancy rate']['1990-01-01':'1999-12-01'], 'o-', label='1990s')
plt.plot(dfq['Unemployment rate']['2000-01-01':'2009-12-01'], 
         dfq['Vacancy rate']['2000-01-01':'2009-12-01'], 'o-', label='2000s')
plt.plot(dfq['Unemployment rate']['2010-01-01':'2019-12-01'], 
         dfq['Vacancy rate']['2010-01-01':'2019-12-01'], 'ko-', label='2010s')

plt.legend(frameon=False)
plt.xlim(2, 12)
plt.ylim(1, 5)
plt.yticks(np.arange(1, 6))

plt.xlabel('Unemployment rate (%)')
plt.ylabel('Vacancy rate (%)')

plt.title('Shifts in the US Beveridge curve')
# plt.savefig('BC.pdf')
plt.show()

Separate trend from cycle to eliminate structural shifts to the Beveridge curve, note the adjustment of filtering options to monthly frequency

In [ ]:
hp_cycle_uv = pd.DataFrame()
hp_trend_uv = pd.DataFrame()

cf_cycle_uv = pd.DataFrame()
cf_trend_uv = pd.DataFrame()

for col in ['Vacancy rate','Unemployment rate']:
    hp_cycle_uv[col], hp_trend_uv[col] = sm.tsa.filters.hpfilter(100*np.log(df[col]).dropna(), lamb=1600*3**4)
    cf_cycle_uv[col], cf_trend_uv[col] = sm.tsa.filters.cffilter(100*np.log(df[col]).dropna(), low=1.5*12, high=8*12)

Plot cyclical components of unemployment and vacancy rates vs cyclical component of output

In [ ]:
fig, ax = plt.subplots()

cf_cycle_uv.resample('QS').mean().to_period('D').plot(ax=ax, style=['k','r'], lw=2)
cf_cycle['Output'].plot(ax=ax, style=['b'], lw=2)

ax.set_ylim(-60, 60)
ylim = ax.get_ylim()

ax.hlines(0, hours.index[0], hours.index[-1], linewidth=0.5)

ax.fill_between(fred.index, ylim[0], ylim[1], fred['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

ax.set_xlim('1950-01', hours.index[-1])

l = ax.legend(loc='upper right')
l.get_frame().set_linewidth(0)
l.get_frame().set_alpha(1)

plt.title('Deviations from Christiano-Fitzgerald trend (%)')
# plt.savefig('VU_CF.pdf')
plt.show()

Run a (very simplified) linear regression on cyclical components of unemployment and vacancy rates, the slope is very close to -1

In [ ]:
x = hp_cycle_uv['Unemployment rate']
y = hp_cycle_uv['Vacancy rate']

slope, intercept = np.polyfit(x, y, 1)

print(slope)
print(intercept)

fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(x, y, alpha=0.25) #facecolor='none', edgecolor='C0'
plt.plot(x, intercept + slope*x, 'r-', lw=2)

plt.xlim(-45, 45)
plt.ylim(-45, 45)

plt.hlines(0, -45, 45, linewidth=0.5)
plt.vlines(0, -45, 45, linewidth=0.5)

plt.title('Deviations from Hodrick-Prescott trend (%)')
plt.xlabel('Unemployment rate')
plt.ylabel('Vacancy rate')

# plt.savefig('BC_HP.pdf')

plt.show()

Generate the 'estimated' Beveridge curve without structural shifts

In [ ]:
u = np.mean(dfq['Unemployment rate'])
v = np.mean(dfq['Vacancy rate'])

print(u, v)

scale = np.linspace(-40, 60, 101)

plt.plot(u*np.exp(scale/100), v*np.exp(slope*scale/100), 'r', lw=2)

plt.plot(u, v, 'ko')

plt.xlim(2, 12)
plt.ylim(1, 5)
plt.yticks(np.arange(1, 6))

# plt.hlines(v, 2, u, linestyle='--', lw=1)
# plt.vlines(u, 1, v, linestyle='--', lw=1)

plt.xlabel('Unemployment rate (%)')
plt.ylabel('Vacancy rate (%)')
plt.title('US Beveridge curve without structural shifts')

# plt.savefig('BC_est.pdf')

plt.show()

In [ ]:
u = np.mean(dfq['Unemployment rate'])
v = np.mean(dfq['Vacancy rate'])

print(u, v)

scale = np.linspace(-40, 60, 101)

plt.plot(u*np.exp(scale/100), v*np.exp(slope*scale/100), 'r', lw=2)
plt.scatter(u*np.exp(x/100), v*np.exp(y/100), alpha=0.25) #, marker='.'

plt.plot(u, v, 'ko')

plt.xlim(2, 12)
plt.ylim(1, 5)
plt.yticks(np.arange(1, 6))

plt.hlines(v, 2, 12, linewidth=0.5)
plt.vlines(u, 1, 5, linewidth=0.5)

plt.xlabel('Unemployment rate (%)')
plt.ylabel('Vacancy rate (%)')
plt.title('US Beveridge curve without structural shifts')

# plt.savefig('BC_est_2.pdf')

plt.show()